In [1]:
%run alpha3.py

In [2]:
# Load questionaire
ua        = loaders.Useralias()
qdf       = pd.read_json('../../allan_data/RGender_.json')
qdf.index = qdf.index.map(lambda x: ua[x])


In [3]:
datafile = datafiles[1][0]
G_org = nx.read_edgelist(datafile, create_using=nx.DiGraph())

In [4]:
q, g, entries_kept = get_aligned_data(qdf.alcohol_binge10__answer, G_org)

In [5]:
entries_kept

0.949685534591195

In [6]:
len(set(g.nodes).intersection(set(q.index.tolist())))

749

In [7]:
sorted(g.nodes) == sorted(q.index.tolist())

True

In [8]:
len(q.index.tolist() )

749

In [9]:
gg = graph.nxDiGraph2Graph(g)
amca = nx.adjacency_matrix(gg)
# np.random.shuffle(amca.data)


In [10]:
for i, el in enumerate(nx.get_edge_attributes(gg, 'weight').items()):
    if i > 10:
        break
    print(el)

(('u0578', 'u0461'), 332.0)
(('u0578', 'u0532'), 17.5)
(('u0578', 'u0578'), 32.0)
(('u0578', 'u0593'), 2.0)
(('u0578', 'u0828'), 159.5)
(('u0578', 'u0847'), 112.0)
(('u0578', 'u0777'), 0.5)
(('u0578', 'u0933'), 2.0)
(('u0578', 'u0809'), 592.5)
(('u0461', 'u0847'), 1629.5)
(('u0461', 'u0593'), 19410.0)


In [11]:
import random
links, weights = zip(*nx.get_edge_attributes(gg, 'weight').items())
weights = list(weights)
random.shuffle(weights)

gg_sh = nx.Graph()
gg_sh.add_weighted_edges_from((u, v, w) for ((u, v), w) in zip(links, weights))
# nx.set_edge_attributes(gg, {(u, v): w for ((u, v), w) in zip(links, weights)}, name='weight')

In [12]:
d1 = np.array(list(nx.get_edge_attributes(gg, 'weight').values()))
d2 = np.array(list(nx.get_edge_attributes(gg_sh, 'weight').values()))
assert np.allclose(d1.mean(), d2.mean())
assert np.allclose(d1.std(), d2.std())
assert (d1 != d2).sum() != len(d1)


In [13]:
gg2 = gg.copy() 
nx.set_edge_attributes(gg2, {(u, v): w for ((u, v), w) in zip(links, weights)}, name='weight')

In [14]:
u = 'u0872'

In [15]:
gg[u]

AtlasView({'u0746': {'weight': 1.5}, 'u0540': {'weight': 45.5}, 'u0683': {'weight': 3.0}, 'u0760': {'weight': 13.5}, 'u0922': {'weight': 7.0}, 'u0390': {'weight': 0.5}, 'u0349': {'weight': 4.0}, 'u0454': {'weight': 7.5}, 'u0773': {'weight': 32.5}, 'u1010': {'weight': 5.0}, 'u0250': {'weight': 32.5}, 'u0675': {'weight': 16.5}, 'u0934': {'weight': 35.0}, 'u0797': {'weight': 0.5}, 'u0887': {'weight': 13.0}, 'u0532': {'weight': 18.0}, 'u0340': {'weight': 44.5}, 'u0138': {'weight': 58.5}, 'u0267': {'weight': 0.5}, 'u0275': {'weight': 1.0}, 'u0500': {'weight': 1.0}, 'u0650': {'weight': 193.0}, 'u0761': {'weight': 1.5}, 'u0872': {'weight': 2.0}, 'u1019': {'weight': 2.0}})

In [16]:
u_weights = np.array([v['weight'] for v in gg[u].values()])
u_weights

array([  1.5,  45.5,   3. ,  13.5,   7. ,   0.5,   4. ,   7.5,  32.5,   5. ,  32.5,  16.5,  35. ,   0.5,  13. ,  18. ,  44.5,  58.5,   0.5,
         1. ,   1. , 193. ,   1.5,   2. ,   2. ])

In [17]:
u_friends = list(gg[u])
u_friends

['u0746',
 'u0540',
 'u0683',
 'u0760',
 'u0922',
 'u0390',
 'u0349',
 'u0454',
 'u0773',
 'u1010',
 'u0250',
 'u0675',
 'u0934',
 'u0797',
 'u0887',
 'u0532',
 'u0340',
 'u0138',
 'u0267',
 'u0275',
 'u0500',
 'u0650',
 'u0761',
 'u0872',
 'u1019']

In [18]:
v = 'u0540'

In [19]:
gg2[u][v]

{'weight': 23.5}

In [20]:
alpha = np.linspace(0, 2, 11)
alpha

array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ])

In [21]:
len(u_friends)

25

In [23]:
tdf = pd.DataFrame(u_weights[:, np.newaxis]**alpha, columns=alpha, index=u_friends)

In [24]:
tdf = tdf / tdf.sum(axis=0)

In [25]:
tdf.shape

(25, 11)

In [26]:
tdf.columns.name = 'alpha'
tdf.index.name = f'{u} friends'

In [27]:
tdf.head()

alpha,0.0,0.2,0.4,0.6000000000000001,0.8,...,1.2000000000000002,1.4000000000000001,1.6,1.8,2.0
u0872 friends,,,,,,,,,,,
u0746,0.04,0.028289,0.017980,0.010397,0.005554,...,0.001319,0.000598,0.000261,0.000111,0.000046
u0540,0.04,0.055977,0.070397,0.080546,0.085146,...,0.079164,0.071052,0.061433,0.051499,0.042098
u0683,0.04,0.032496,0.023725,0.015758,0.009671,...,0.003030,0.001579,0.000792,0.000386,0.000183
u0760,0.04,0.043901,0.043300,0.038854,0.032212,...,0.018421,0.012967,0.008793,0.005781,0.003706
u0922,0.04,0.038497,0.033296,0.026200,0.019047,...,0.008376,0.005170,0.003074,0.001772,0.000996


In [28]:
tdf.loc[v]

alpha
0.0    0.040000
0.2    0.055977
0.4    0.070397
0.6    0.080546
0.8    0.085146
1.0    0.084337
1.2    0.079164
1.4    0.071052
1.6    0.061433
1.8    0.051499
2.0    0.042098
Name: u0540, dtype: float64

In [37]:
if 'w_ij' not in gg2[u][v]:
    print("Setting w_ij")
    gg2[u][v]['w_ij'] = tdf.loc[v]
if 'w_ij' in gg2[v][u]:
    print("It's bidirectional properties in the graph.")

It's bidirectional properties in the graph.


In [38]:
seen = set()

In [39]:
seen.add(frozenset((u, v)))

In [42]:
seen

{frozenset({'u0540', 'u0872'})}

In [41]:
{u, v} in seen

True